<a href="https://colab.research.google.com/github/noelzogbladan/sharezza/blob/master/Donn%C3%A9es_scraping_modmob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installer et importer les librairies

In [ ]:
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
pip install --upgrade --force-reinstall pandas

     |████████████████████████████████| 9.9MB 22.1MB/s 
     |████████████████████████████████| 235kB 39.5MB/s 
     |████████████████████████████████| 15.3MB 333kB/s 
     |████████████████████████████████| 512kB 41.2MB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Found existing installation: numpy 1.19.5
    

In [ ]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in /usr/local/lib/python3.7/dist-packages (1.2.3)


In [ ]:
pip install beautifulsoup4

In [ ]:
pip install gtfstk

     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 10.4MB 27.6MB/s 
     |████████████████████████████████| 10.5MB 31.3MB/s 
  Created wheel for utm: filename=utm-0.4.2-cp37-none-any.whl size=5422 sha256=db991d8b51dcf2ece48c393e949b52e60dc904ed3cc2c216069a485ee74755f8
  Stored in directory: /root/.cache/pip/wheels/1d/2d/5a/4c6320941f607ffefd353e1bb715e33a250a59f835cf0fbdd8
Successfully built utm
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 0.25.3 which is incompatible.
  Found existing installation: pandas 1.2.3
    Uninstalling pandas-1.2.3:
      Successfully uninstalled pandas-1.2.3


In [ ]:
pip install requests

In [ ]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
import time
import urllib.request
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
#import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  del sys.path[0]


#Données Bourgogne-Franche-Comté

Définir une fonction de recupération de toutes les pages

In [ ]:
def get_link_pages(url_bus, nb):
    pages = []
    for i in range(1,nb+1):
        j=url_bus+"?page="+str(i)
        pages.append(j)
    return pages


In [ ]:
pages_FC=get_link_pages("https://transport.data.gouv.fr/datasets/region/1",2)
pages_FC

['https://transport.data.gouv.fr/datasets/region/1?page=1',
 'https://transport.data.gouv.fr/datasets/region/1?page=2']

Définir une fonction de recupération des liens de toutes les pages de la région

In [ ]:
def lien_pages_region(lien_region):
  page=[]
  for url in lien_region:
    page2=requests.get(url)
    soup1= BeautifulSoup(page2.text,"html.parser")
    for i in range(31,38):# Recupérer les url de tous les réseaux de transport dans la région
      data=soup1.findAll('a')[i]
      if data.text.lstrip("\n'")=="'Réseau urbain Optymo":
        break
      link=data['href']
      download_url='https://transport.data.gouv.fr/'+link # composition des urls des pages des réseaux
    #url_to_use= '"'+download_url+'"'
      page.append(download_url)#url_to_use)
  return page
  time.sleep(1)

Définir une fonction de rcupéartion de toutes les datasets de la région

In [ ]:
def liens_dataset(url_pages):
  page_1=[]
  for j in url_pages:
    page_=requests.get(j)
    soup_= BeautifulSoup(page_.text,"html.parser")
    for i in range(len(soup_.findAll('a'))):
      data_1=soup_.findAll('a')[i]["href"]
      if "geojson" in data_1:
        page_1.append(data_1)
  #page_1["href"]
        liens_data= list(set(page_1))
  return liens_data
        #print(liens_page)
  time.sleep(1)

Définir une fonction de concatenation des datasets

In [ ]:
def concat_data(liens):
  appended_data = []
  for lien in liens:
    data = gpd.read_file(lien)
    # store DataFrame in list
    appended_data.append(data)
# see pd.concat documentation for more info
  return gpd.GeoDataFrame(pd.concat( appended_data, ignore_index=True))
# write DataFrame to an excel sheet 
#appended_data.to_excel('appended.xlsx')
  #return appended_data

In [ ]:
pages_FC=get_link_pages("https://transport.data.gouv.fr/datasets/region/1",2)
region_pages= lien_pages_region(pages_FC)
datasest_region=liens_dataset(region_pages)
data_bus_FC=concat_data(datasest_region)

In [ ]:
data_bus_FC["Région"]=["Bourgogne Franche Comté" for i in range(len(data_bus_FC))]

In [ ]:
data_bus_FC.isnull().sum()

code                   14523
description             2316
id                      2316
name                    2316
parent_station         14891
wheelchair_boarding     2316
geometry                 812
route_color            22378
route_id               22378
route_long_name        22378
route_short_name       22378
route_text_color       22455
Région                     0
dtype: int64

In [ ]:
data_bus_FC.isnull().sum()

code                   14523
description             2316
id                      2316
name                    2316
parent_station         14891
wheelchair_boarding     2316
geometry                 812
route_color            22378
route_id               22378
route_long_name        22378
route_short_name       22378
route_text_color       22455
Région                     0
dtype: int64

In [ ]:
for url in pages_FC:
  page2=requests.get(url)
  soup1= BeautifulSoup(page2.text,"html.parser")
  for i in range(31,38):# Recupérer les url de tous les réseaux de transport dans la région
      data=soup1.findAll('a')[i]
      if data.text.lstrip("\n'")=="'Réseau urbain Optymo":
        break
    #if "region" in data:
      print(data)

<a href="/datasets/reseau-de-transports-en-commun-de-la-communaute-dagglomeration-de-lauxerrois/">
Réseau urbain Léo                          </a>
<a href="/datasets/horaires-theoriques-du-reseau-vbus-communaute-dagglomeration-de-vesoul-gtfs/">
Réseau urbain Vbus                          </a>
<a href="/datasets/offre-de-transport-reseau-tallis-gtfs/">
Réseau urbain Tallis                          </a>
<a href="/datasets/horaires-du-reseau-de-bus-intercom-2/">
Réseau interurbain Intercom                          </a>
<a href="/datasets/reseau-de-transport-du-grand-dole/">
Réseau urbain TGD                          </a>
<a href="/datasets/gtfs-diviamobilites/">
Réseau urbain DiviaMobilités                          </a>
<a href="/datasets/offre-de-transport-de-la-c-a-beaune-cote-sud-gtfs/">
Réseau urbain Côte&amp;Bus                           </a>
<a href="/datasets/reseau-de-transport-interurbain-mobigo-en-bourgogne-franche-comte/">
Réseau interurbain Mobigo                          </a>

In [ ]:
soup1.findAll('a')[37].text.lstrip("\n'")

'Réseau urbain Optymo                          '

In [ ]:
data

<a href="/datasets/arrets-circuits-et-horaires-du-reseau-optymo-de-belfort/">
Réseau urbain Optymo                          </a>

In [ ]:
link= data['href']

In [ ]:
link

'/datasets/arrets-circuits-et-horaires-du-reseau-optymo-de-belfort/'

In [ ]:
urls_test=lien_pages_region(pages_FC)

In [ ]:
#page=[]
#for url in pages_FC:
 # page2=requests.get(url)
  #soup1= BeautifulSoup(page2.text,"html.parser")
  #for i in range(31,38):# Recupérer les url de tous les réseaux de transport dans la région
   #   data=soup1.findAll('a')[i]
     # if data.text.lstrip("\n'")=="'Réseau urbain Optymo":
    #    break
      #link=data['href']
      #download_url='https://transport.data.gouv.fr/'+link # composition des urls des pages des réseaux
    #url_to_use= '"'+download_url+'"'
      #page.append(download_url)#url_to_use)
      #print(page)
      #time.sleep(1)



In [ ]:
#liens_dataset(url_pages)
#page_1=[]
#for j in df_pages["Les liens"]:
#  page_=requests.get(j)
#  soup_= BeautifulSoup(page_.text,"html.parser")
 # for i in range(len(soup_.findAll('a'))):
  #  data_1=soup_.findAll('a')[i]["href"]
   # if "geojson" in data_1:
    #  page_1.append(data_1)
  #page_1["href"]
 #     liens_page=list(set(page_1))
  #    print(liens_page)
   #   time.sleep(1)

Définir une fonction unzip

In [ ]:
import tempfile
import zipfile
import gtfstk as gt

In [ ]:
def download_unzip(url, dirname = tempfile.gettempdir(), destname = "borders"):
  myfile = requests.get(url)
  open(dirname + '/' + destname + '.zip', 'wb').write(myfile.content)
  with zipfile.ZipFile(dirname + '/' + destname + '.zip', 'r') as zip_ref:
      zip_ref.extractall(dirname + '/' + destname)

In [ ]:
#download_unzip(url)
#communes = gpd.read_file(temporary_location + "/borders/communes-20210101.shp")
#communes.head()

Définir une fonction de concatenation les dataframes

In [ ]:
#locator = Nominatim(user_agent="myGeocoder", timeout=10)
#rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [ ]:
#tqdm_notebook.pandas()
#appended_data['address'] = appended_data['geometry'].progress_apply(rgeocode)
#df.head()

#Région numero 2 Rhone Alpes

In [ ]:
pages_RA=get_link_pages("https://transport.data.gouv.fr/datasets/region/2",4)
for url in pages_RA:
  page2=requests.get(url)
  soup1= BeautifulSoup(page2.text,"html.parser")
  for i in range(32,37):# Recupérer les url de tous les réseaux de transport dans la région
      data=soup1.findAll('a')[i]
      if data.text.lstrip("\n'")=="'Réseau urbain Proxim iTi":
        break
    #if "region" in data:
      #print(data)

Fonction liens région

In [ ]:
def lien_pages_regionRA(lien_region):
  page=[]
  for url in lien_region:
    page2=requests.get(url)
    soup1= BeautifulSoup(page2.text,"html.parser")
    for i in range(32,37):# Recupérer les url de tous les réseaux de transport dans la région
      data=soup1.findAll('a')[i]
      if data.text.lstrip("\n'")=="'Réseau urbain proxim iti":
        break
      link=data['href']
      download_url='https://transport.data.gouv.fr/'+link # composition des urls des pages des réseaux
    #url_to_use= '"'+download_url+'"'
      page.append(download_url)#url_to_use)
  return page
  time.sleep(1)

Dataset

In [ ]:
pages_RA=get_link_pages("https://transport.data.gouv.fr/datasets/region/2",4)
region_pagesRA= lien_pages_regionRA(pages_RA)
datasest_regionRA=liens_dataset(region_pagesRA)
data_bus_RA=concat_data(datasest_regionRA)

In [ ]:
data_bus_RA["Région"]=["Auvergne Rhone Alpes" for i in range(len(data_bus_RA))]

In [ ]:
data_bus_RA.shape

(21377, 13)

In [ ]:
data_bus_RA.isnull().sum()

description             1186
id                      1186
name                    1186
wheelchair_boarding     1186
code                   14767
parent_station         10856
geometry                   1
route_id               20223
route_long_name        20223
route_short_name       20223
route_color            20767
route_text_color       20849
Région                     0
dtype: int64

#Région Centre Val de Loire

In [ ]:
pages_VL=get_link_pages("https://transport.data.gouv.fr/datasets/region/3",2)
for url in pages_VL:
  page2=requests.get(url)
  soup1= BeautifulSoup(page2.text,"html.parser")
  for i in range(31,39):# Recupérer les url de tous les réseaux de transport dans la région
      data=soup1.findAll('a')[i]
      if data.text.lstrip("\n'")=="'Réseau urbain Proxim iTi":
        break
    #if "region" in data:
      print(data)

<a href="/datasets/gtfs-tao-kmo/">
Réseau urbain TAO                          </a>
<a href="/datasets/offre-de-transports-urbain-du-reseau-amelys-gtfs/">
Réseau urbain Amelys                          </a>
<a href="/datasets/agglopolys-offre-theorique-mobilite-reseau-urbain-azalys-de-blois/">
Réseau urbain Agglopolys                          </a>
<a href="/datasets/dreux-offre-theorique-mobilite-reseau-urbain/">
Réseau urbain Linead                          </a>
<a href="/datasets/agglobus-offre-theorique-mobilite-reseau-urbain-de-bourges/">
Réseau urbain AggloBus                          </a>
<a href="/datasets/offre-de-transport-du-reseau-move-ca-territoires-vendomois-gtfs/">
Réseau urbain Move                          </a>
<a href="/datasets/filibus-periurbain/">
Réseau péri-urbain Filibus - Hiver 2019-2020                          </a>
<a href="/datasets/issoudun-offre-theorique-mobilite-reseau-urbain/">
Réseau urbain Transport Issoudun Gratuit                          </a>
<a href=

Fonction de liens de la région

In [ ]:
def lien_pages_regionVL(lien_region):
  page=[]
  for url in lien_region:
    page2=requests.get(url)
    soup1= BeautifulSoup(page2.text,"html.parser")
    for i in range(len(soup1.findAll('a'))):# Recupérer les url de tous les réseaux de transport dans la région
      if "Réseau" in soup1.findAll('a')[i].text.lstrip("\n'"):
        data=soup1.findAll('a')[i]
      #if data.text.lstrip("\n'")=="'Réseau urbain proxim iti":
        #break
        link=data['href']
        download_url='https://transport.data.gouv.fr/'+link # composition des urls des pages des réseaux
    #url_to_use= '"'+download_url+'"'
        page.append(download_url)#url_to_use)
  return page
  time.sleep(1)

In [ ]:
pages_VL=get_link_pages("https://transport.data.gouv.fr/datasets/region/3",2)
region_pagesVL= lien_pages_regionVL(pages_VL)
datasest_regionVL=liens_dataset(region_pagesVL)
data_bus_VL=concat_data(datasest_regionVL)

In [ ]:
data_bus_VL["Région"]=["Centre Val de Loire" for i in range(len(data_bus_VL))]

In [ ]:
data_bus_VL.shape

(39332, 14)

In [ ]:
data_bus_VL.isnull().sum()

description             1388
id                      1388
name                    1388
parent_station         17814
timezone               28341
wheelchair_boarding     1388
geometry                  36
route_color            37984
route_id               37944
route_long_name        37944
route_short_name       37944
route_text_color       37984
code                   37304
Région                     0
dtype: int64

#Région Grand Est

In [ ]:
url_GE="https://transport.data.gouv.fr/datasets/region/4"
pages_GE=get_link_pages(url_GE,2)
for url in pages_GE:
  page2=requests.get(url)
  soup1= BeautifulSoup(page2.text,"html.parser")
  for i in range(len(soup1.findAll('a'))):# Recupérer les url de tous les réseaux de transport dans la région
        if "Réseau" in soup1.findAll('a')[i].text.lstrip("\n'"):
          data=soup1.findAll('a')[i]
          if data.text.lstrip("\n'")=="'Réseau urbain Proxim iTi":
            break
    #if "region" in data:
        #print(data)

In [ ]:
def lien_pages_region(lien_region):
  page=[]
  for url in lien_region:
    page2=requests.get(url)
    soup1= BeautifulSoup(page2.text,"html.parser")
    for i in range(len(soup1.findAll('a'))):# Recupérer les url de tous les réseaux de transport dans la région
      if "Réseau" in soup1.findAll('a')[i].text.lstrip("\n'"):
        data=soup1.findAll('a')[i]
      #if data.text.lstrip("\n'")=="'Réseau urbain proxim iti":
       # break
        link=data['href']
        download_url='https://transport.data.gouv.fr/'+link # composition des urls des pages des réseaux
    #url_to_use= '"'+download_url+'"'
        page.append(download_url)#url_to_use)
  return page
  time.sleep(1)

In [ ]:
pages_GE=get_link_pages("https://transport.data.gouv.fr/datasets/region/4",6)
region_pagesGE= lien_pages_region(pages_GE)
datasest_regionGE=liens_dataset(region_pagesGE)
data_bus_GE=concat_data(datasest_regionGE)

In [ ]:
data_bus_GE["Région"]=["Grand Est" for i in range(len(data_bus_GE))]

In [ ]:
data_bus_GE.shape

(36448, 13)

In [ ]:
data_bus_GE.isnull().sum()

description             1781
id                      1781
name                    1781
wheelchair_boarding     1781
geometry                   1
parent_station         27848
route_color            34865
route_id               34667
route_long_name        34667
route_short_name       34667
route_text_color       34679
code                   27417
Région                     0
dtype: int64

#Région Haut de France

In [ ]:
pages_HF=get_link_pages("https://transport.data.gouv.fr/datasets/region/5",3)
region_pagesHF= lien_pages_region(pages_HF)
datasest_regionHF=liens_dataset(region_pagesHF)
data_bus_HF=concat_data(datasest_regionHF)

In [ ]:
data_bus_HF.shape

(57097, 12)

In [ ]:
data_bus_HF["Région"]=["Haut de France" for i in range(len(data_bus_HF))]

In [ ]:
data_bus_HF.isnull().sum()

code                   25119
description             6234
id                      6234
name                    6234
wheelchair_boarding     6234
route_color            51773
route_id               50863
route_long_name        50863
route_short_name       50863
route_text_color       51773
geometry                   9
parent_station         41463
Région                     0
dtype: int64

#Région Ile de France

In [ ]:
pages_IF=get_link_pages("https://transport.data.gouv.fr/datasets/region/6",2)
region_pagesIF= lien_pages_region(pages_IF)
datasest_regionIF=liens_dataset(region_pagesIF)
data_bus_IF=concat_data(datasest_regionIF)

In [ ]:
data_bus_IF.shape

(63940, 11)

In [ ]:
data_bus_IF["Région"]=["Ile de France" for i in range(len(data_bus_IF))]

In [ ]:
data_bus_IF.isnull().sum()

description               19
id                        19
name                      19
wheelchair_boarding       19
route_color            63921
route_id               63921
route_long_name        63921
route_short_name       63921
route_text_color       63921
geometry                   0
parent_station         19307
Région                     0
dtype: int64

#Région de Bretagne

In [ ]:
pages_BR=get_link_pages("https://transport.data.gouv.fr/datasets/region/7",2)
region_pagesBR= lien_pages_region(pages_BR)
datasest_regionBR=liens_dataset(region_pagesBR)
data_bus_BR=concat_data(datasest_regionBR)

In [ ]:
data_bus_BR.shape

(58447, 12)

In [ ]:
data_bus_BR["Région"]=["Bretagne" for i in range(len(data_bus_BR))]

In [ ]:
data_bus_BR.isnull().sum()

description             9645
id                      9645
name                    9645
wheelchair_boarding     9645
route_color            48802
route_id               48802
route_long_name        48802
route_short_name       48802
route_text_color       48802
geometry                   0
code                   18992
parent_station         53264
Région                     0
dtype: int64

#Région de Normandie

In [ ]:
pages_NO=get_link_pages("https://transport.data.gouv.fr/datasets/region/8",2)
region_pagesNO= lien_pages_region(pages_NO)
datasest_regionNO=liens_dataset(region_pagesNO)
data_bus_NO=concat_data(datasest_regionNO)

In [ ]:
data_bus_NO.shape

(18092, 12)

In [ ]:
data_bus_NO["Région"]=["Normandie" for i in range(len(data_bus_NO))]

In [ ]:
data_bus_NO.isnull().sum()

code                    3664
description              108
id                       108
name                     108
wheelchair_boarding      108
geometry                  18
parent_station          7261
route_color            17984
route_id               17984
route_long_name        17984
route_short_name       17984
route_text_color       17984
Région                     0
dtype: int64

#Région Nouvelle acquitaine

In [ ]:
pages_NA=get_link_pages("https://transport.data.gouv.fr/datasets/region/9",7)
region_pagesNA= lien_pages_region(pages_NA)
datasest_regionNA=liens_dataset(region_pagesNA)
data_bus_NA=concat_data(datasest_regionNA)

In [ ]:
data_bus_NA.shape

(70995, 12)

In [ ]:
data_bus_NA["Région"]=["Nouvelle Acquitaine" for i in range(len(data_bus_NA))]

In [ ]:
data_bus_NA.isnull().sum()

code                   53119
description             5109
id                      5109
name                    5109
wheelchair_boarding     5109
parent_station         33122
route_color            65900
route_id               65886
route_long_name        65886
route_short_name       65886
route_text_color       65990
geometry                   0
Région                     0
dtype: int64

#Région Pays de la Loire

In [ ]:
pages_PL=get_link_pages("https://transport.data.gouv.fr/datasets/region/10",3)
region_pagesPL= lien_pages_region(pages_PL)
datasest_regionPL=liens_dataset(region_pagesPL)
data_bus_PL=concat_data(datasest_regionPL)

In [ ]:
data_bus_PL.shape

(59100, 12)

In [ ]:
data_bus_PL["Région"]=["Pays de la Loire" for i in range(len(data_bus_PL))]

In [ ]:
data_bus_PL.isnull().sum()

description            12173
id                     12173
name                   12173
wheelchair_boarding    12173
code                   24443
parent_station         34835
geometry                   0
route_color            46929
route_id               46929
route_long_name        46929
route_short_name       46929
route_text_color       46929
Région                     0
dtype: int64

#Corse

In [ ]:
pages_CO=get_link_pages("https://transport.data.gouv.fr/datasets/region/11",1)
region_pagesCO= lien_pages_region(pages_CO)
datasest_regionCO=liens_dataset(region_pagesCO)
data_bus_CO=concat_data(datasest_regionCO)

In [ ]:
data_bus_CO.shape

(2207, 12)

In [ ]:
data_bus_CO["Région"]=["Corse" for i in range(len(data_bus_CO))]

In [ ]:
data_bus_CO.isnull().sum()

code                    573
description             498
id                      498
name                    498
wheelchair_boarding     498
parent_station         1255
route_color            1709
route_id               1709
route_long_name        1709
route_short_name       1709
route_text_color       1709
geometry                  0
Région                    0
dtype: int64

#Haute-Garonne

In [ ]:
pages_HG=get_link_pages("https://transport.data.gouv.fr/datasets/region/12",4)
region_pagesHG= lien_pages_region(pages_HG)
datasest_regionHG=liens_dataset(region_pagesHG)
data_bus_HG=concat_data(datasest_regionHG)

In [ ]:
data_bus_HG.shape

(64868, 13)

In [ ]:
data_bus_HG["Région"]=["Occitanie" for i in range(len(data_bus_HG))]

In [ ]:
#data_bus_HG.name=="Capitole"
#data_bus_HG.name.index('Capitole')
for i,e in enumerate(data_bus_HG["name"]):
  if e == 'Mermoz':
     print (i,e)

5027 Mermoz
5064 Mermoz
5971 Mermoz
6210 Mermoz
22115 Mermoz
22470 Mermoz
24766 Mermoz
25081 Mermoz
30587 Mermoz
30926 Mermoz
32038 Mermoz
32812 Mermoz
32961 Mermoz
34203 Mermoz
35260 Mermoz
35670 Mermoz
35988 Mermoz
43212 Mermoz
43306 Mermoz
44612 Mermoz
45202 Mermoz
59341 Mermoz
59948 Mermoz


In [ ]:
data_bus_HG.isnull().sum()

code                   29161
description             8480
id                      8480
name                    8480
wheelchair_boarding     8480
geometry                 541
route_id               56388
route_long_name        56388
route_short_name       56388
parent_station         45287
route_color            58558
route_text_color       59654
timezone               62851
Région                     0
dtype: int64

#Région Province Alpe côte d'Azur 

In [ ]:
pages_PA=get_link_pages("https://transport.data.gouv.fr/datasets/region/13",2)
region_pagesPA= lien_pages_region(pages_PA)
datasest_regionPA=liens_dataset(region_pagesPA)
data_bus_PA=concat_data(datasest_regionPA)

In [ ]:
data_bus_PA.shape

(50190, 12)

In [ ]:
data_bus_PA["Région"]=["PACA" for i in range(len(data_bus_PA))]

In [ ]:
data_bus_PA.isnull().sum()

description             2079
id                      2079
name                    2079
wheelchair_boarding     2079
parent_station         38340
route_color            48144
route_id               48111
route_long_name        48111
route_short_name       48111
route_text_color       49120
geometry                   0
code                   35996
Région                     0
dtype: int64

#La Réunion

In [ ]:
pages_RU=get_link_pages("https://transport.data.gouv.fr/datasets/region/15",1)
region_pagesRU= lien_pages_region(pages_RU)
datasest_regionRU=liens_dataset(region_pagesRU)
data_bus_RU=concat_data(datasest_regionRU)

In [ ]:
data_bus_RU.shape

(17051, 10)

In [ ]:
data_bus_RU["Région"]=["La Réunion" for i in range(len(data_bus_RU))]

In [ ]:
data_bus_RU.isnull().sum()

description              102
id                       102
name                     102
wheelchair_boarding      102
geometry                1217
code                   12715
parent_station         12868
route_id               16949
route_long_name        16949
route_short_name       16949
Région                     0
dtype: int64

#Guadeloupe

In [ ]:
pages_GD=get_link_pages("https://transport.data.gouv.fr/datasets/region/16",1)
region_pagesGD= lien_pages_region(pages_GD)
datasest_regionGD=liens_dataset(region_pagesGD)
data_bus_GD=concat_data(datasest_regionGD)

In [ ]:
data_bus_GD.shape

(2069, 6)

In [ ]:
data_bus_GD["Région"]=["Guadeloupe" for i in range(len(data_bus_GD))]

In [ ]:
data_bus_GD.isnull().sum()

description              0
id                       0
name                     0
parent_station         739
wheelchair_boarding      0
geometry                 0
Région                   0
dtype: int64

#Guyane

In [ ]:
pages_GU=get_link_pages("https://transport.data.gouv.fr/datasets/region/19",1)
region_pagesGU= lien_pages_region(pages_GU)
datasest_regionGU=liens_dataset(region_pagesGU)
data_bus_GU=concat_data(datasest_regionGU)

In [ ]:
data_bus_GU.shape

(1123, 12)

In [ ]:
data_bus_GU["Région"]=["Guyane" for i in range(len(data_bus_GU))]

In [ ]:
data_bus_GU.isnull().sum()

code                   621
description            547
id                     547
name                   547
wheelchair_boarding    547
parent_station         833
geometry                 0
route_color            576
route_id               576
route_long_name        576
route_short_name       576
route_text_color       576
Région                   0
dtype: int64

#Dataset Général France

Concatenation des datasets

In [ ]:
df_bus=gpd.GeoDataFrame(pd.concat((data_bus_FC[["name","geometry","Région"]],data_bus_RA[["name","geometry","Région"]],data_bus_NO[["name","geometry","Région"]],data_bus_PL[["name","geometry","Région"]],data_bus_HG[["name","geometry","Région"]],data_bus_NA[["name","geometry","Région"]],data_bus_IF[["name","geometry","Région"]],data_bus_PA[["name","geometry","Région"]],data_bus_GE[["name","geometry","Région"]],data_bus_CO[["name","geometry","Région"]],data_bus_GD[["name","geometry","Région"]]),ignore_index=True))

In [ ]:
X=data_bus_RU[["name","geometry","Région"]],data_bus_VL[["name","geometry","Région"]],data_bus_BR[["name","geometry","Région"]],data_bus_HF[["name","geometry","Région"]]

In [ ]:
df_bus_general=gpd.GeoDataFrame(pd.concat((df_bus,data_bus_RU[["name","geometry","Région"]],data_bus_VL[["name","geometry","Région"]],data_bus_BR[["name","geometry","Région"]],data_bus_HF[["name","geometry","Région"]]),ignore_index=True))

In [ ]:
df_bus_general.shape

(585907, 3)

In [ ]:
df_bus_general.isnull().sum()

name        51118
geometry     2635
Région          0
dtype: int64

In [ ]:
df_bus_general["Région"].unique()

array(['Bourgogne Franche Comté', 'Auvergne Rhone Alpes', 'Normandie',
       'Pays de la Loire', 'Occitanie', 'Nouvelle Acquitaine',
       'Ile de France', 'PACA', 'Grand Est', 'Corse', 'Guadeloupe',
       'La Réunion', 'Centre Val de Loire', 'Bretagne', 'Haut de France'],
      dtype=object)

Nettoyage de la dataset générale

In [ ]:
index_with_nan=df_bus_general.index[df_bus_general.isnull().any(axis=1)]
index_with_nan

Int64Index([     9,     18,     20,     22,     26,     30,     31,     32,
                35,     39,
            ...
            585574, 585575, 585576, 585577, 585578, 585579, 585580, 585581,
            585582, 585583],
           dtype='int64', length=53744)

In [ ]:
df_bus_general_sans_nan = df_bus_general.drop(index_with_nan,0)


In [ ]:
df_bus_general_sans_nan.isnull().sum()

name        0
geometry    0
Région      0
dtype: int64

In [ ]:
df_bus_general_sans_nan.shape

(532163, 3)

*Supprimer les doublons*

In [ ]:
df_bus_general_final=df_bus_general_sans_nan.drop_duplicates(subset ="geometry", keep = 'first')#, inplace=True)


In [ ]:
df_bus_general_final.shape

(270419, 3)

In [ ]:
for i,e in enumerate(df_bus_general_final["name"]):
  if e == 'Le Lac Reynerie':
     print (i,e)

70937 Le Lac Reynerie
74578 Le Lac Reynerie
74735 Le Lac Reynerie


In [ ]:
df_bus_general_final.geometry.type.unique

<bound method Series.unique of 0         Point
1         Point
2         Point
3         Point
4         Point
          ...  
585344    Point
585354    Point
585373    Point
585392    Point
585395    Point
Length: 270419, dtype: object>

Génération des adresses des arrêts

In [ ]:
from shapely.geometry import Point


Extraire les coordonnées long et lat 

In [ ]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x|y') of edges/vertices of a Polygon/others"""

    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        for ea in geometry:
            if coord_type == 'x':
                all_xy.append(list( ea.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.xy[1] ))
        return all_xy

    if geometry.type=='MultiPolygon':
        all_xy = []
        for ea in geometry:
            if coord_type == 'x':
                all_xy.append(list( ea.exterior.coords.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.exterior.coords.xy[1] ))
        return all_xy

    else:
        # Finally, return empty list for unknown geometries
        return []

In [ ]:
# Get the Polygon x and y coordinates
df_bus_general_final['x'] = df_bus_general_final.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
df_bus_general_final['y'] = df_bus_general_final.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4213: FutureWarning: 'is_extension_type' is deprecated and will be removed in a future version.  Use 'is_extension_array_dtype' instead.
  if callable(index) or is_dict_like(index):
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
df_bus_general_final.isnull().sum()

name        0
geometry    0
Région      0
x           0
y           0
dtype: int64

In [ ]:
print(df_bus_general_final.head(3))

                       name  ...                      geom
0  VESOUL Capitaine Blandin  ...  [47.6335338],[6.1458318]
1          FROTEY Rue Haute  ...   [47.6226476],[6.186741]
2          NAVENNE Calmette  ...  [47.6074043],[6.1616041]

[3 rows x 6 columns]


Reverse code pour générer les adresses

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import pandas as pd
#import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
df_bus_general_final["geom"] = df_bus_general_final["y"].map(str) + "," + df_bus_general_final["x"].map(str)
df_bus_general_final["geom"][0]

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


'[47.6335338],[6.1458318]'

In [ ]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [ ]:
tqdm_notebook.pandas()
df_bus_general_final['address'] = df_bus_general_final['geom'].progress_apply(rgeocode)
#df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/geopy/extra/rate_limiter.py", line 130, in __call__
    return self.func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py", line 369, in reverse
    self._call_geocoder(url, timeout=timeout), exactly_one
  File "/usr/local/lib/python3.7/dist-packages/geopy/geocoders/base.py", line 362, in _call_geocoder
    raise ERROR_CODE_MAP[code](message)
geopy.exc.GeocoderQueryError: HTTP Error 400: Bad Request
RateLimiter caught an error, retrying (1/2 tries). Called with (*('[47.6187515],[6.190236]',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "

#Généralisation

Définir une fonction de recupération des liens

In [ ]:
def get_pages(url_bus, nb):
    pages = []
    for i in range(1,nb+1):
        j = url_bus + str(i)
        for k in range(1,10):
          l=j+"?page="+str(k)
          pages.append(l)
    return pages
   # https://transport.data.gouv.fr/datasets/region/1?page=1

La page princiapale du scraping pour bus

In [ ]:
url_bus="https://transport.data.gouv.fr/datasets/region/"

Liens pour toutes les régions

In [ ]:
Liens=get_pages(url_bus,19)
Liens

In [ ]:
for lien in Liens:
  response=requests.get(lien)
  soup_gen= BeautifulSoup(response.text,"html.parser")
  page_gen=[]
  for i in range(31,38):
    data_gen=soup_gen.findAll('a')[i]["href"]
    download_url_gen='https://transport.data.gouv.fr/'+data_gen
    #url_to_use= '"'+download_url+'"'
    page_gen.append(download_url_gen)#url_to_use)
    print(page_gen)
    time.sleep(1) 
    


In [ ]:
df_gen = pd.DataFrame(page_gen, columns=['Les liens'])
df_gen

In [ ]:
page_gen1=[]
for j in page_gen:
      response1=requests.get(j)
      soup_gen1= BeautifulSoup(response1.text,"html.parser")
      for i in range(len(soup_gen1.findAll('a'))):
          data_gen1=soup_gen1.findAll('a')[i]["href"]
          if "json" in data_gen1:
            page_gen1.append(data_gen1)
  #page_1["href"]
            print(page_gen1)
            time.sleep(1)
#df_pages_gen=pd.DataFrame(page_gen, columns=['Les liens'])

In [ ]:
df_gen1=pd.DataFrame(page_gen1,columns=["Les liens"])
df_gen1

In [ ]:
for 
    page_gen=[]
    for i in range(30,41):
      data_gen1= soup1.findAll('a')[i]
    #if "datasets" in data:
      link_gen=data_gen1['href']
      download_url_gen='https://transport.data.gouv.fr/'+link
    #url_to_use= '"'+download_url+'"'
      page_gen.append(download_url_gen)#url_to_use)
    #if "region" in data:

In [ ]:
#page1=[]
#for i in range(1, 10):#len(soup_.findAll('a'))+1):
#    data_=soup_.findAll('a')[i]
#    link_=data_['href']
    #download_url1='https://transport.data.gouv.fr/'+link
    #url_to_use= '"'+download_url+'"'
 #   page1.append(link)#url_to_use)
  #lien=data_['href']
#    print(page1)

In [ ]:
response_bus=requests.get("https://transport.data.gouv.fr/datasets?q=arrets+bus")
soup_bus= BeautifulSoup(response_bus.text,"html.parser")

In [ ]:
page_bus=[]
for i in range(57):
    data_bus= soup_bus.findAll('a')[i]
    #if "datasets" in data:
    #link_bus=data_bus['href']
    download_url_bus='https://transport.data.gouv.fr/'+link_bus
    #url_to_use= '"'+download_url+'"'
    page_bus.append(download_url_bus)#url_to_use)

In [ ]:
page_bus

In [ ]:
df_bus = pd.DataFrame(page_bus, columns=['Les liens'])
df_bus

In [ ]:
page_bus1=[]
for j in page_bus:#df_bus["Les liens"]:
  response_bus1=requests.get(j)
  soup_bus1= BeautifulSoup(response_bus1.text,"html.parser")
  for i in range(65):
    data_bus1=soup_bus1.findAll('a')[i]["href"]
    if "geojson" in data_bus1:
      page_bus1.append(data_bus1)
  #page_1["href"]
      liens_page_bus=list(set(page_bus1))
      print(liens_page_bus)
      time.sleep(1)

In [ ]:
df_bus1 = pd.DataFrame(liens_page_bus, columns=['Les liens'])
df_bus1

In [ ]:
b=[]
for i in range(5):
  a=i+4
  b.append(a)
  print(b)

In [ ]:
A=pd.DataFrame(b,columns=["i"])
A

In [ ]:
for i in A["i"]:
  print(i)

#Codes à explorer

In [ ]:
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

pages = np.arange(1, 1001, 50)

for page in pages: 
  
  page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt", headers=headers)
  #https://transport.data.gouv.fr/datasets/region/1
  
  soup = BeautifulSoup(page.text, 'html.parser')
  
  movie_div = soup.find_all('div', class_='lister-item mode-advanced')
  
  sleep(randint(2,10))